<a href="https://colab.research.google.com/github/Tomcruzzz/AEY/blob/master/scrape_stock_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 42.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.6 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.3 MB/s eta 0:00:00


In [3]:
from transformers import PegasusTokenizer, PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

model = AutoModelForSeq2SeqLM.from_pretrained("human-centered-summarization/financial-summarization-pegasus")

In [5]:
monitored_tickers = ['DOGE', 'ETH', 'BTC']

In [6]:
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs 

In [7]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls

{'DOGE': ['/?sa=X&ved=0ahUKEwi336mSo8L9AhXz5XMBHUKSAuAQOwgC',
  '/search?q=yahoo+finance+DOGE&tbm=nws&ie=UTF-8&gbv=1&sei=YTkDZPeqCPPLz7sPwqSKgA4',
  '/search?q=yahoo+finance+DOGE&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwi336mSo8L9AhXz5XMBHUKSAuAQ_AUIBSgA',
  '/search?q=yahoo+finance+DOGE&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwi336mSo8L9AhXz5XMBHUKSAuAQ_AUIBygC',
  '/search?q=yahoo+finance+DOGE&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwi336mSo8L9AhXz5XMBHUKSAuAQ_AUICCgD',
  'https://maps.google.com/maps?q=yahoo+finance+DOGE&um=1&ie=UTF-8&sa=X&ved=0ahUKEwi336mSo8L9AhXz5XMBHUKSAuAQ_AUICSgE',
  '/search?q=yahoo+finance+DOGE&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwi336mSo8L9AhXz5XMBHUKSAuAQ_AUICigF',
  '/search?q=yahoo+finance+DOGE&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwi336mSo8L9AhXz5XMBHUKSAuAQ_AUICygG',
  '/advanced_search',
  '/search?q=yahoo+finance+DOGE&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X&ved=0ahUKEwi336mSo8L9AhXz5XMBHUKSAuAQpwUIDQ',
  '/search?q=yahoo+finance

In [8]:
import re

In [9]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [10]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [11]:

cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], exclude_list) for ticker in monitored_tickers}
cleaned_urls

{'DOGE': ['https://uk.finance.yahoo.com/news/elon-musk-dogecoin-tweet-crypto-shiba-inu-bitcoin-ethereum-144528544.html',
  'https://finance.yahoo.com/news/elon-musk-recommends-investing-physical-181902373.html',
  'https://finance.yahoo.com/news/elon-musk-dog-added-500-193533317.html',
  'https://finance.yahoo.com/news/bit-mining-limited-announces-unaudited-110000243.html',
  'https://sg.finance.yahoo.com/news/cryptoverse-elon-musk-frees-bird-061151493.html',
  'https://finance.yahoo.com/news/long-traders-bear-brunt-bitcoin-093357009.html',
  'https://www.google.com/search?q%3Dyahoo%2Bfinance%2BDOGE%26tbm%3Dnws%26pccc%3D1',
  'https://uk.finance.yahoo.com/news/doge-surges-speculation-elon-musk-ethereum-vitalik-buterin-working-together-110407259.html',
  'https://finance.yahoo.com/news/couch-famous-doge-meme-auctioned-140000470.html',
  'https://uk.finance.yahoo.com/news/dogecoin-crypto-doge-elon-musk-bitcoin-ethereum-104915583.html',
  'https://au.finance.yahoo.com/news/tesla-stock-fal

In [12]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs: 
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [13]:

articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'DOGE': ['Tax changes coming in to force this April could make a big difference to your savings and investments Elon Musk posted a cryptic tweet on Wednesday that saw dogeoin (DOGE-USD) gain whilst the rest of the cryptocurrency market slumped after Tuesday\'s broad sell-off on US markets. Dogecoin increased in value after the Tesla (TSLA) boss tweeted: "High time I confessed I let the Doge out". Read more: Crypto live prices The tweet saw dogecoin\'s value spike 0.1%, a 2.8% rise in the past week, as the rest of the cryptocurrency market turned red. Before Musk\'s tweet, dogecoin was down 3% on the day, trading at $0.082. But since his cryptic message the memecoin rose to $0.086. In comparison, bitcoin (BTC-USD) has fallen from its early week high of above $25,000 to $24,170, a drop of 2% in the past 24 hours. Ethereum (ETH-USD) also fell 2% in the past day, to $1,646, as of the time of writing. On Wednesday, the global cryptocurrency market cap stood at $1.15tn, down 2.0% in the pas

In [14]:

def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [15]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'DOGE': ['Musk posts cryptic tweet: “High time I confessed I let the Doge out”.',
  'Physical assets have historically performed well during high inflation.',
  'Dogecoin, Floki, and Shiba Inu all surged after Musk’s post. Tesla is making ‘merch buyable’ with Dogecoin',
  'Financial results for the fourth quarter and full year ended December 31, 2022.',
  'We are aware of the issue and are working to resolve it.',
  'Kraken settles SEC charges with $30 million payment. Long trades take 90% of liquidated crypto futures',
  'All images are copyrighted.',
  'We are aware of the issue and are working to resolve it.',
  'PleasrDAO is holding a 24-hour auction for Kabosu’s couch. The charity chosen by Atsuko and the Own The Doge community',
  'We are aware of the issue and are working to resolve it.',
  'We are aware of the issue and are working to resolve it.'],
 'ETH': ['Musk posts cryptic tweet: “High time I confessed I let the Doge out”.',
  'I wish I could be there, but I’m still recov

In [16]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [17]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

{'DOGE': [{'label': 'NEGATIVE', 'score': 0.987929105758667},
  {'label': 'POSITIVE', 'score': 0.9995119571685791},
  {'label': 'POSITIVE', 'score': 0.9934555888175964},
  {'label': 'NEGATIVE', 'score': 0.6459313631057739},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.983890950679779},
  {'label': 'NEGATIVE', 'score': 0.9880996346473694},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'NEGATIVE', 'score': 0.9723355174064636},
  {'label': 'POSITIVE', 'score': 0.9979088306427002},
  {'label': 'POSITIVE', 'score': 0.9979088306427002}],
 'ETH': [{'label': 'NEGATIVE', 'score': 0.987929105758667},
  {'label': 'POSITIVE', 'score': 0.968347430229187},
  {'label': 'NEGATIVE', 'score': 0.9879298210144043},
  {'label': 'POSITIVE', 'score': 0.9893380403518677},
  {'label': 'POSITIVE', 'score': 0.9281173944473267},
  {'label': 'NEGATIVE', 'score': 0.9994637370109558},
  {'label': 'NEGATIVE', 'score': 0.9992609620094299},
  {'label': 'NEG

In [18]:
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this = [
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [19]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output

[['DOGE',
  'Musk posts cryptic tweet: “High time I confessed I let the Doge out”.',
  'NEGATIVE',
  0.987929105758667,
  'https://uk.finance.yahoo.com/news/elon-musk-dogecoin-tweet-crypto-shiba-inu-bitcoin-ethereum-144528544.html'],
 ['DOGE',
  'Physical assets have historically performed well during high inflation.',
  'POSITIVE',
  0.9995119571685791,
  'https://finance.yahoo.com/news/elon-musk-recommends-investing-physical-181902373.html'],
 ['DOGE',
  'Dogecoin, Floki, and Shiba Inu all surged after Musk’s post. Tesla is making ‘merch buyable’ with Dogecoin',
  'POSITIVE',
  0.9934555888175964,
  'https://finance.yahoo.com/news/elon-musk-dog-added-500-193533317.html'],
 ['DOGE',
  'Financial results for the fourth quarter and full year ended December 31, 2022.',
  'NEGATIVE',
  0.6459313631057739,
  'https://finance.yahoo.com/news/bit-mining-limited-announces-unaudited-110000243.html'],
 ['DOGE',
  'We are aware of the issue and are working to resolve it.',
  'POSITIVE',
  0.99790

In [20]:
final_output.insert(0, ['Ticker', 'Summary', 'Label', 'Confidence', 'URL'])

In [21]:
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)

# New Section